### 特征编码

[参考](https://blog.csdn.net/hshuihui/article/details/53259710)

category 怎么处理
数值型特征怎么处理.

目前比较熟悉的有pandas里的get_dummies做one-hot encoding.

sklearn里也有很多特征编码方法.

下面的例子是要把数值型或者字符型都用sklearn 来one-hot encoding

In [101]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import * 
import numpy as np

dataset = pd.DataFrame({'pet': ['cat', 'dog', 'dog', 'fish'],                         
'age': [4 , 6, 3, 3],                        
'salary':[4, 5, 1, 1],
'lvl':[4, 5, pd.np.nan, 1]})

In [97]:
dataset.head()

,pet,age,salary,lvl
0,cat,4,4,4.0
1,dog,6,5,5.0
2,dog,3,1,NaN
3,fish,3,1,1.0


#### get_dummies 

get_dummies对数值型也是有效果的

In [26]:
dataset.head()

,pet,age,salary,lvl
0,cat,4,4,4.0
1,dog,6,5,5.0
2,dog,3,1,NaN
3,fish,3,1,1.0


In [30]:
d =  pd.get_dummies(dataset,columns=['age','salary','pet','lvl']);
d.head()

##nan treat as [0,0....0]


,age_3,age_4,age_6,salary_1,salary_4,salary_5,pet_cat,pet_dog,pet_fish,lvl_1.0,lvl_4.0,lvl_5.0
0,0,1,0,0,1,0,1,0,0,0,1,0
1,0,0,1,0,0,1,0,1,0,0,0,1
2,1,0,0,1,0,0,0,1,0,0,0,0
3,1,0,0,1,0,0,0,0,1,1,0,0


dummy_na 就是把na也编码

In [31]:
d =  pd.get_dummies(dataset,columns=['age','salary','pet','lvl'],dummy_na=True);
d.head()

,age_3.0,age_4.0,age_6.0,age_nan,salary_1.0,salary_4.0,salary_5.0,salary_nan,pet_cat,pet_dog,pet_fish,pet_nan,lvl_1.0,lvl_4.0,lvl_5.0,lvl_nan
0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0
1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0
2,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
3,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0


#### sklearn里的用法 输入2D-arrayt, 得到的是也是2Dnarray  比pd麻烦多了
* 字符型不能处理
*  Nan不能处理

In [58]:

age= OneHotEncoder(sparse = False).fit_transform( dataset[['age']])
# pet= OneHotEncoder(sparse = False).fit_transform( dataset[['pet']])
salary= OneHotEncoder(sparse = False).fit_transform( dataset[['salary']])
# lvl = OneHotEncoder(sparse = False).fit_transform( dataset[['lvl']])
d = np.hstack( (age,salary))

(4, 6)

LabelEncoder 针对字符型label. ['a','b','c']--->[0,1,2]
先LabelEncoder ,再OneHotEncoder才能达到最终的效果

In [66]:
numi_pet = LabelEncoder().fit_transform(dataset[['pet']])
print(numi_pet)
pet =  OneHotEncoder(sparse = False).fit_transform( numi_pet.reshape(-1,1))
pet

[0 1 1 2]


/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

等效于LabelBinarizer

In [68]:
pet =  LabelBinarizer().fit_transform( numi_pet.reshape(-1,1))
pet

array([[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

### DataFrameMapper

[参考](https://blog.csdn.net/hshuihui/article/details/53260922)
pandas处理dataframe, sklearn很多是以narray为输入.

DataFrameMapper来自sklearn_pandas，可以将sklearn 的 transformer灵活地运用在  DataFrame 当中

DataFrameMapper内部机制是先将指定的 DataFrame 的列转换成 ndarray 类型，再输入到 sklearn 的相应 transformer中

`DataFrameMapper就是提前定义好mapper,针对输入X的某些列，根据mapper做相应的数据变换`

对列的变换比较灵活，可筛选出一个或多个列，并用一个或多个 sklearn 的 transformer 作用，组合起来极其强大；同时通过继承机制，它本身也可以看做是 sklearn 的 transformer 类，输入 sklearn 的相关类(如Pipeline, FeatureUnion)


In [111]:
import random 
import sklearn 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn_pandas import DataFrameMapper, cross_val_score
# frameworks for ML from sklearn_pandas import DataFrameMapper from sklearn.pipeline import make_pipeline from sklearn.cross_validation import cross_val_score from sklearn.grid_search import GridSearchCV 
# transformers for category variables from sklearn.preprocessing import LabelBinarizer from sklearn.preprocessing import MultiLabelBinarizer from sklearn.preprocessing import LabelEncoder from sklearn.preprocessing import OneHotEncoder 
# transformers for numerical variables from sklearn.preprocessing import MinMaxScaler from sklearn.preprocessing import StandardScaler from sklearn.preprocessing import Normalizer 
# transformers for combined variables from sklearn.decomposition import PCA from sklearn.preprocessing import PolynomialFeatures 
# user-defined transformers from sklearn.preprocessing import FunctionTransformer 
# classification models from sklearn.ensemble import RandomForestClassifier from sklearn.linear_model import LogisticRegression 
# evaluation from sklearn.metrics import scorer 

In [112]:
dataset = pd.DataFrame({'pet': ['cat', 'dog', 'dog', 'fish'],                         
'age': [4 , 6, 3, 3],                        
'salary':[4, 5, 1, 1]})

In [113]:
mapper = DataFrameMapper([        (['age'],[ MinMaxScaler(), StandardScaler()]),    ]) 
mapper.fit_transform(dataset)

/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-1.35973996e-16],
       [ 1.63299316e+00],
       [-8.16496581e-01],
       [-8.16496581e-01]])

In [120]:
mapper = DataFrameMapper([ ('pet', LabelBinarizer()),  (['age'], [MinMaxScaler()]) , (['salary'], OneHotEncoder())    ])

In [123]:
d = mapper.fit_transform(dataset)
d.shape # 7  = 3+ 1+ 3 

/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


(4, 7)

In [ ]:
可以看到mapper还是有用的，尺度归一化，PCA降维,生成二次项特征，都可以很简易的完成

In [125]:
from sklearn.decomposition import PCA
mapper = DataFrameMapper([        
    (['salary','age'], [MinMaxScaler(), PCA(2)]),       
    (['salary','age'],[MinMaxScaler(), PolynomialFeatures(2)])          ]) 
mapper.fit_transform(dataset)

/opt/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.23146041,  0.20995792,  1.        ,  0.75      ,  0.33333333,
         0.5625    ,  0.25      ,  0.11111111],
       [ 0.86453897, -0.11585794,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ],
       [-0.54799969, -0.04704999,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [-0.54799969, -0.04704999,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ]])